In [ ]:
#pre-processing

import re    # for regular expressions 
import nltk  # for text manipulation 
import string 
import warnings 
import numpy as np 
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
all_stopwords_gensim = STOPWORDS

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

#Remove \n
dataset['cleanText'] = np.vectorize(remove_pattern)(dataset['text'], '\n')
#Removing Punctuations and special charachers
dataset['cleanText'] = dataset['cleanText'].str.replace("[^a-zA-Z]", " ") 
#removing short words
dataset['cleanText'] = dataset['text'].apply(
    lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#removing stop words
j=0
for all in dataset['cleanText']:
  dataset['cleanText'][j] = remove_stopwords(all)
  j = j+1

#Tokenizing
tokenized_text = dataset['cleanText'].apply(lambda x: x.split())
#Text Normalization
from nltk.stem.porter import * 
stemmer = PorterStemmer()
tokenized_text = tokenized_text.apply(lambda x: [stemmer.stem(i) for i in x])

for i in range(len(tokenized_text)):
    tokenized_text[i] = ' '.join(tokenized_text[i])
dataset['cleanText'] = tokenized_text

In [ ]:
dataset.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

data=[]
for x in dataset['cleanText']:
  for all in x.split():
    data.append(all)

#BOW
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=300, 
                                 stop_words='english') 
bow = bow_vectorizer.fit_transform(data)
bow.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=300, 
                                   stop_words='english') 
tfidf = tfidf_vectorizer.fit_transform(data) 
tfidf.shape

In [ ]:
#Word2Vec
model_w2v = gensim.models.Word2Vec(
            data,
            size=300, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2,
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            seed = 34) 
model_w2v.train(data, total_examples= len(tokenized_text), epochs=20)

def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary continue
            if count != 0:
                vec /= count
    return vec

wordvec_arrays = np.zeros((len(tokenized_text), 300)) 
for i in range(len(tokenized_text)):
    wordvec_arrays[i,:] = word_vector(tokenized_text[i], 300)
    wordvec_df = pd.DataFrame(wordvec_arrays) 
wordvec_df.shape 

In [ ]:
!pip install tqdm
from tqdm import tqdm  
from gensim.models.doc2vec import LabeledSentence

def add_label(twt):
  output = []
  for i, s in zip(twt.index, twt):
    output.append(LabeledSentence(s, ["tweet_"+str(i)]))
  return output

tokenized_tweet = dataset['cleanText'].apply(lambda x: x.split()) # tokenizing 
labeled_tweets = add_label(dataset['cleanText'])  
print(labeled_tweets[:1])

model_d2v = gensim.models.Doc2Vec(
            dm=1,
            dm_mean=1,
            size=300, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2,
            hs = 0,
            negative = 10, # for negative sampling
            workers= 2, # no.of cores
            alpha=0.1,
            seed = 34) 

model_d2v.build_vocab([i for i in tqdm(labeled_tweets)])

model_d2v.train(labeled_tweets, total_examples= len(data), epochs=20)

doc2vec_arrays = np.zeros((len(tokenized_tweet), 300)) 
for i in range(len(tokenized_tweet)):
    doc2vec_arrays[i,:] = model_d2v.docvecs[i].reshape((1, 300))
    docvec_df = pd.DataFrame(doc2vec_arrays) 
docvec_df.shape 